In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


In [2]:
import string
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.corpus import stopwords
from subprocess import check_output
from sklearn.feature_extraction.text import CountVectorizer

In [83]:
import re

In [104]:
pd.set_option('display.max_colwidth', -1)

In [2]:
chan = pd.read_csv("https://drive.google.com/file/d/1VlIWC77lKy6AON_hXITdyS7N186EMjRR/view?usp=sharing")
chan = chan[chan.Char == "Chandler"].Diag[::20]

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 2


In [129]:
chan

2         \nAll right Joey, be nice.  \n\nSo does he have a hump? A hump and a hairpiece?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
127       Oh, uh, that would be mine. See, I wrote a note to myself, \n\nand then I realised I didn't need it, so I balled it up and... (sees that Monica \n\nis glaring at him) ...now I wish I was dead.                                   

In [130]:
def clean_text(txt):
    txt = "".join(txt.split('\n'))
    txt = re.sub("\(.*?\)","",txt)
    txt = re.sub("\[.*?\]","",txt)
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in chan]
corpus[:15]

[' all right joey be nice so does he have a hump a hump and a hairpiece',
 ' oh uh that would be mine see i wrote a note to myself and then i realised i didnt need it so i balled it up and  now i wish i was dead ',
 ' all right im putting it out im putting it out  ',
 ' or you can hold it in your mouth and smoke it like this ',
 ' its just you and rachel just the two of you this is a date youre going on a date',
 '  she said yes she said yes  awful play man whoah  her names aurora and shes italian and she pronounces my name chandlrr chandlrr i think i like it better that way  oh listen the usher gave me this to give to you ',
 ' oh so what youre saying is youre not completely fulfilled by rick ethan and myself',
 ' huh did uh any of the rest of you guys think that when you first met me',
 ' no i prefer to keep a safe distance from all this merriment',
 ' alright janice thats it janice janice hey janice when i invited you to this party i didnt necessarily think that it meant that we',
 

In [131]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]


[[43, 46],
 [43, 46, 73],
 [43, 46, 73, 20],
 [43, 46, 73, 20, 116],
 [43, 46, 73, 20, 116, 18],
 [43, 46, 73, 20, 116, 18, 155],
 [43, 46, 73, 20, 116, 18, 155, 54],
 [43, 46, 73, 20, 116, 18, 155, 54, 21],
 [43, 46, 73, 20, 116, 18, 155, 54, 21, 5],
 [43, 46, 73, 20, 116, 18, 155, 54, 21, 5, 156]]

In [132]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [133]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 110, 10)           7130      
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 713)               72013     
Total params: 123,543
Trainable params: 123,543
Non-trainable params: 0
_________________________________________________________________


In [134]:


model.fit(predictors, label, epochs=100, verbose=5)



Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [143]:

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
#         print(output_word)
    return seed_text.title()


In [150]:
print(generate_text("president",100,model,max_sequence_len))

President I Dont Just Talking To Have The Wedding Time Last Night She Waslying To Our Faces Of Me And And The Scores Oh About And I Think It The Dream Who Eats His Own Face You Ifigure Ill Be Crazy Man Man Ill Getmore Snakes Call Them My Babies Kids Will Walk Past My Place They Will Runrun Away From Crazy Snake Man Theyll Shout Man Theyll Shout Man My Babies Kids Will Walk Past My Place They Will Runrun Away From Crazy Snake Man Theyll Shout Man Theyll Shout Man My Babies Kids Will Walk Past My Place They


In [ ]:
1. Learning more advanced Machine Learning Algorithms.
2. Implementing them to handle more complex     data-sets and improving accuracy on the current ones.
3. Exploring a sub field of ML ,Natural Language Processing .
4. Start working on the project.

In [ ]:
1. Learned advanced ML algorithms : Random-Forest, SVM, Naive-Bayes, Neural-networks.
2. These increased the already high accuracies.
3. Worked on data-set having, prices of houses in California and attributes of the houses such as : coordinates, age, distance from the ocean etc.
4. Started learning Natural Language Processing.  

In [ ]:
1. Learn more NLP concepts.
2. Work on a data-set involving spam/ham prediction of text messages.

In [ ]:
    >>> Tokenization    
    >>> Stemming
    >>> Lemmatization 
    >>> False Negative
    >>> Accuracy
    >>> Precision
    >>> Recall
    >>> Classification Threshold